### Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time # to be used to delay
import re
import glob
import requests
import json

In [2]:
# executable_path = {'executable_path':'resources/chromedriver'}
executable_path = {'executable_path':ChromeDriverManager().install()}

# create a new browser object by default it is firefox the default one
browser = Browser('chrome', **executable_path, headless = False) #headless true will not open browser

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [C:\Users\gucas\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


### Mass Shootings

In [3]:
# Empty dataframe to append tables
combined_msa = pd.DataFrame()

msa_url = 'https://www.gunviolencearchive.org/mass-shooting'
browser.visit(msa_url)
for i in range(81):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(2)
    msa_df = pd.read_html(html)
    time.sleep(2)
    msa_df= msa_df[1]
    time.sleep(2)
    #If no Next button found, exit loop
    try:
        browser.click_link_by_partial_text("next")
    except:
        break
    
    # Append all dataframes
    combined_msa = combined_msa.append(msa_df)

    # Export to csv
    filepath = f"data/source/msa{i + 1}.csv"
    msa_df.to_csv(filepath, index=False)
    print(f"Successfully scraped table from page {i + 1}")
    
# Reset index before exporting
combined_msa = combined_msa.reset_index(drop=True)

# Export combined dataframes
combined_msa.to_csv('data/combined_msa.csv', index=False)

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


Successfully scraped table from page 1
Successfully scraped table from page 2
Successfully scraped table from page 3
Successfully scraped table from page 4
Successfully scraped table from page 5
Successfully scraped table from page 6
Successfully scraped table from page 7
Successfully scraped table from page 8
Successfully scraped table from page 9
Successfully scraped table from page 10
Successfully scraped table from page 11
Successfully scraped table from page 12
Successfully scraped table from page 13
Successfully scraped table from page 14
Successfully scraped table from page 15
Successfully scraped table from page 16
Successfully scraped table from page 17
Successfully scraped table from page 18
Successfully scraped table from page 19
Successfully scraped table from page 20
Successfully scraped table from page 21
Successfully scraped table from page 22
Successfully scraped table from page 23
Successfully scraped table from page 24
Successfully scraped table from page 25
Successfu

In [4]:
# Close browser
browser.quit()

In [5]:
# Set column Incident Date to date format
combined_msa["Incident Date"] = pd.to_datetime(combined_msa["Incident Date"])
combined_msa.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,1993551,2021-05-03,Michigan,Saginaw,1315 Randolph St,1,4,View Incident View Source
1,1993090,2021-05-02,Ohio,Columbus,491 Park St,0,4,View Incident View Source
2,1993664,2021-05-02,Pennsylvania,Philadelphia,5200 block of Burton St,1,3,View Incident View Source
3,1993221,2021-05-02,Maryland,Baltimore,1500 block of Washington Blvd,1,3,View Incident View Source
4,1993280,2021-05-02,Illinois,Chicago,2100 block of W Adams St,0,5,View Incident View Source


In [6]:
clean_msa = combined_msa.drop(['Operations'], axis =1)
clean_msa.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured
0,1993551,2021-05-03,Michigan,Saginaw,1315 Randolph St,1,4
1,1993090,2021-05-02,Ohio,Columbus,491 Park St,0,4
2,1993664,2021-05-02,Pennsylvania,Philadelphia,5200 block of Burton St,1,3
3,1993221,2021-05-02,Maryland,Baltimore,1500 block of Washington Blvd,1,3
4,1993280,2021-05-02,Illinois,Chicago,2100 block of W Adams St,0,5


In [7]:
clean_msa = clean_msa.rename(columns = {'Incident ID':'incident_id','Incident Date':'incident_date', 'State':'state', 'City Or County':'city_county',
                         'Address':'address', '# Killed':'number_killed', '# Injured':'number_injured'})
clean_msa.head()

,incident_id,incident_date,state,city_county,address,number_killed,number_injured
0,1993551,2021-05-03,Michigan,Saginaw,1315 Randolph St,1,4
1,1993090,2021-05-02,Ohio,Columbus,491 Park St,0,4
2,1993664,2021-05-02,Pennsylvania,Philadelphia,5200 block of Burton St,1,3
3,1993221,2021-05-02,Maryland,Baltimore,1500 block of Washington Blvd,1,3
4,1993280,2021-05-02,Illinois,Chicago,2100 block of W Adams St,0,5


In [8]:
# Check dataset
len(clean_msa)

1975

In [9]:
# Check for unique values
len(clean_msa["incident_id"].unique())

1975

In [ ]:
# # Drop duplicates
# clean_msa = clean_msa.drop_duplicates(subset = 'incident_id', keep = "first")

In [ ]:
# # Dataset after dropping duplicates
# len(clean_msa["incident_id"].unique())

In [ ]:
# # Export clean dataset
# clean_msa.to_csv('data/clean_msa.csv', index=False)

# Google API

In [ ]:
# address = "1701 10th Ave S, Great Falls, Montana"
# gkey = "Enter Google API Key"
# base_url = "https://maps.googleapis.com/maps/api/geocode/json"
# params = {
#     "address": address, 
#     "key": gkey
# }
# response = requests.get(base_url, params=params)
# response_json = response.json()
# lat = response_json["results"][0]["geometry"]["location"]["lat"]
# lng = response_json["results"][0]["geometry"]["location"]["lng"]
# print(response.url)
# print(f"{lat}, {lng}")

In [10]:
# Function to get latitude and longitude using a API call
def get_latlng(df):
    df['latitude'] = ""
    df['longitude'] = ""
    gkey = "Enter Google API Key"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    for index, row in df.iterrows():
        street = row['address']
        city = row['city_county']
        state = row['state']
        address = f"{street}, {city}, {state}"
        params = {
            "address": address, 
            "key": gkey
    }
    #     target_address = "Seattle, Washington"
        response = requests.get(base_url, params=params)
#         time.sleep(2)
        response_json = response.json()
        try:
            lat = response_json["results"][0]["geometry"]["location"]["lat"]
            lng = response_json["results"][0]["geometry"]["location"]["lng"]
            
            df.loc[index, 'latitude'] = lat
            df.loc[index, 'longitude'] = lng
        except:
            pass
    return df

In [11]:
final_msa = get_latlng(clean_msa)
final_msa.head(20)

,incident_id,incident_date,state,city_county,address,number_killed,number_injured,latitude,longitude
0,1993551,2021-05-03,Michigan,Saginaw,1315 Randolph St,1,4,43.4033,-83.9476
1,1993090,2021-05-02,Ohio,Columbus,491 Park St,0,4,39.9722,-83.0052
2,1993664,2021-05-02,Pennsylvania,Philadelphia,5200 block of Burton St,1,3,40.015,-75.071
3,1993221,2021-05-02,Maryland,Baltimore,1500 block of Washington Blvd,1,3,39.279,-76.643
4,1993280,2021-05-02,Illinois,Chicago,2100 block of W Adams St,0,5,41.8788,-87.6795
5,1992570,2021-05-02,Minnesota,Saint Paul,843 Selby Ave,0,4,44.9468,-93.1358
6,1992127,2021-05-01,Delaware,Wilmington,800 block of N Pine St,0,4,39.7409,-75.5429
7,1992534,2021-05-01,Louisiana,New Orleans,St Bernard Ave and N Claiborne Ave,2,4,29.9726,-90.0655
8,1992410,2021-05-01,New York,New Rochelle,Horton Ave and Brook St,1,4,40.9199,-73.7891
9,1992490,2021-05-01,Massachusetts,Holyoke,209 South St,0,5,42.1959,-72.6218


In [12]:
# Check dataset
len(final_msa)

1975

In [13]:
# Drop empty rows
final_msa = final_msa.dropna(how='any')

In [14]:
# Check dataset after dropping rows
len(final_msa)

1972

In [15]:
# Reset index before exporting
final_msa = final_msa.reset_index(drop=True)

In [16]:
# Export final csv file
final_msa.to_csv("data/final_msa.csv")